# Feature detection

During the feature detection we retrieve all the postal codes and split them into four seperate numbers per postal code. We grayscale the image and extract features. At last we export the features to a csv file.

In [140]:
# imports

from SimpleCV import *
# from IPython.display import HTML
import matplotlib.pyplot as plt
import pandas as pd
# import cv2
import glob

### Image subtraction and splitting

- We get the images from the folder. We binarize, dilate, and erode to respectively make them black and white to make the number stand out, to make the letters smoother and thicker, and to remove noise. Next we invert them again to make the numbers black and the background white and embiggen the images. We save the image and the inverted image with the correct label.
- next we split the postal code in four seperate images of 32x32

In [141]:
disp = Display(displaytype='notebook')
size = 32

imagelist = []
horizontalSplit=[]

# get all files and add them to tuple with the postal codes
for filename in glob.glob("../dataset-images/*.png"): 
    originalImage = Image(filename)
    image = originalImage.binarize().dilate().erode()
    invertedImage = image.invert().embiggen( (128,40), Color.WHITE, (3,3))
    answer = filename[18:22]
    imagelist.append((answer, image, invertedImage))

numbers = []
   
for image in imagelist:
    for i in range(4):
        sizeHeight = 40
        sizeWidth = 32
        digit = image[0][(0 if i+1 == 1 else i):i+1]
        normalImage = image[1].crop(i*size, 0, size, size*(i+1))
        invertedImage = image[2].crop(i*sizeWidth, 0, sizeWidth, sizeHeight*(i+1))
        numbers.append((digit, normalImage, invertedImage))

## Functions for feature extraction

- createMatrix - divide the pictures in pieces of 8x8. *crop.getNumpy* get's a numpy array of the pixels and takes one of the three rgb colours. Because they are grayscale it doesn't matter which one we take. Divide by 255 to see if it's one or zero. one is white and zero is black. Next add up all the 0's and 1's and you get the amount of white pixels in a 8x8 grid. The white pixels are the numbers and the black pixels are the background.
- isCentroidPixelBlack - checks if the centroid - weighted center - is black. This means that the center of the number should be empty. If the number is 0 for example, the middle pixel should be black.
- getBlob - finds the blob from the image and returns this.
- centroidTopBottom - split's the number in half horizontaly and takes the centroid as dividing center. Then checks the centroid of the image and returns 1 if it's black. For example with an 8, the centroid of the two halfs should both be black.

In [149]:
# divide number images into pieces of 4x4 and create matrix. input image is 32X32
def createMatrix(image):
    # size of cropped image
    size = 4
    m = np.zeros((8,8))

    # loop trough 8x8
    for x in range(8):
        for y in range(8):
            # crop image to 4x4
            crop = image.crop(x*size, y*size, size, size)
            # get numpy array with colour and divide by 255
            pixels = crop.getNumpy()[:,:,0] / 255
            # count the filled pixels
            m[x][y] = pixels.sum()
    return m.T.astype(int)

In [144]:
# crop image to x and y of centroid and get the centroid pixel and the mean color
# check if centroid pixel is above the threshold. If above return 1 for true else 0 
def isCentroidPixelBlack(image, centroid):
    img = image.crop(centroid[0], centroid[1], 1, 1)
    color = img.meanColor()
    return 0 if color[0] > 200 else 1

In [148]:
# get the blob from the image and return
def getBlob(image):
    image.show()
    blobs = image.findBlobs()
    image.draw(blobs, autocolor=True)
    return blobs[0]

# divide image horizontaly, get the centroid and return if they are black or not
def centroidTopBottom(image, centroidY, height):    
    croppedImageTop = image.crop(0,0,height,centroidY)
    blobTop = getBlob(croppedImageTop)
    centroidTop = blobTop.centroid()
    pixelBlackTop = isCentroidPixelBlack(croppedImageTop,centroidTop)
    
    croppedImageBottom = image.crop(0,centroidY, height,(height-centroidY))   
    blobBottom = getBlob(croppedImageBottom)
    centroidBottom = blobBottom.centroid()
    pixelBlackBottom = isCentroidPixelBlack(croppedImageBottom,centroidBottom)

    return (pixelBlackTop, pixelBlackBottom)

## Creating the feature list

First we define the dictionary *numbersDict*. Next we will go through all the individual numbers in the list and add the features to *numbersDict*. We then add the columns to the pandas dataframe and add the filled dictionary. Lastly we will reorder the dataframe *df* to make sure the labels will be the last column

In [165]:
# numbersDict = {
#     "blob_area" : [],
#     "blob_centroid_x" : [],
#     "blob_centroid_y" : [],
#     "blob_angle" : [],
#     "blob_height": [],
#     "blob_width": [],
#     "blob_amount_contours": [],
#     "number_of_holes":[],
#     "centroid_is_empty": [],
#     "centroid_top_is_empty":[],
#     "centroid_bottom_is_empty":[],
#     "label" : []
# }

# for x in range(8):
#     for y in range(8):
#         numbersDict["grid_" + str(x) + "_" + str(y)] = []

    
# for number in numbers:
#     digit = number[0]
#     image = number[1]
#     invertedImage = number[2]
#     invertedBlobCount = len(invertedImage.findBlobs()) -1
#     blob = getBlob(image)
#     matrix = createMatrix(image)
#     centroid = blob.centroid()
#     split = centroidTopBottom(image, centroid[1], blob.height())
#     for x in range(8):
#         for y in range(8):
#             numbersDict["grid_" + str(x) + "_" + str(y)].append(matrix[y][x])
#     numbersDict["blob_area"].append(blob.area())
#     numbersDict["blob_centroid_x"].append(centroid[0])
#     numbersDict["blob_centroid_y"].append(centroid[1])
#     numbersDict["blob_angle"].append(blob.angle())
#     numbersDict["blob_height"].append(blob.height())
#     numbersDict["blob_width"].append(blob.width())
#     numbersDict["blob_amount_contours"].append(len(blob.contour()))
#     numbersDict["number_of_holes"].append(invertedBlobCount)
#     numbersDict["centroid_is_empty"].append(isCentroidPixelBlack(image, centroid))
#     numbersDict["centroid_top_is_empty"].append(split[0])
#     numbersDict["centroid_bottom_is_empty"].append(split[1])
#     numbersDict["label"].append(number[0])    

# df = pd.DataFrame(numbersDict)
# columns = ["blob_area", 
#          "blob_centroid_x", 
#          "blob_centroid_y",
#          "blob_angle",
#          "blob_height",
#          "blob_width",
#          "blob_amount_contours",
#          "number_of_holes",
#          "centroid_is_empty",
#          "centroid_top_is_empty",
#          "centroid_bottom_is_empty"]

# # add grid data
# for x in range(8):
#     for y in range(8):
#         columns.append("grid_" + str(x) + "_" + str(y))
        
# # add label last
# columns.append("label")
# df = df[columns]

# # sort by label
# df = df.sort_values("label")

In [178]:
# define dictionary
numbersDict = {
    "blob_area" : [],
    "blob_centroid_x" : [],
    "blob_centroid_y" : [],
    "blob_angle" : [],
    "blob_height": [],
    "blob_width": [],
    "blob_amount_contours": [],
    "number_of_holes":[],
    "centroid_is_empty": [],
    "centroid_top_is_empty":[],
    "centroid_bottom_is_empty":[],
    "label" : []
}

# add grid elements to dictionary
for x in range(8):
    for y in range(8):
        numbersDict["grid_" + str(x) + "_" + str(y)] = []
        
# loop through all the numbers and add the features to the dictionary
for number in numbers:
    # the label
    digit = number[0]
    # the image
    image = number[1]
    # the inverted image
    invertedImage = number[2]
    # find the blobs of inverted image and subtract 1, because this is the background blob. 
    # we do this to see how many holes the number has. A 9 should have one hole and an 8 should have two holes.
    invertedBlobCount = len(invertedImage.findBlobs()) -1
    blob = getBlob(image)
    matrix = createMatrix(image)
    centroid = blob.centroid()
    split = centroidTopBottom(image, centroid[1], blob.height())
    
    for x in range(8):
        for y in range(8):
            numbersDict["grid_" + str(x) + "_" + str(y)].append(matrix[y][x])
    numbersDict["blob_area"].append(blob.area())
    numbersDict["blob_centroid_x"].append(centroid[0])
    numbersDict["blob_centroid_y"].append(centroid[1])
    numbersDict["blob_angle"].append(blob.angle())
    numbersDict["blob_height"].append(blob.height())
    numbersDict["blob_width"].append(blob.width())
    numbersDict["blob_amount_contours"].append(len(blob.contour()))
    numbersDict["number_of_holes"].append(invertedBlobCount)
    numbersDict["centroid_is_empty"].append(isCentroidPixelBlack(image, centroid))
    numbersDict["centroid_top_is_empty"].append(split[0])
    numbersDict["centroid_bottom_is_empty"].append(split[1])
    numbersDict["label"].append(number[0])    

# create dataframe
df = pd.DataFrame(numbersDict)
temp = list(df)

# move label to end of list
temp.insert(len(temp)-1, temp.pop(temp.index('label')))

# use loc to reorder columns
df = df.loc[:, temp]

# sort by label
df = df.sort_values("label")

In [179]:
# export to csv
df.to_csv("../dataset-numpy/dataset_features_contours_test_2.csv", sep=',', encoding='utf-8', index=False)

In [166]:
#####move to feature analysis ####

df.describe()

blob_amount_contours   blob_angle    blob_area  blob_centroid_x  \
count           1920.000000  1920.000000  1920.000000      1920.000000   
mean              44.952083    50.857330   291.653385        15.859298   
std                8.744703    65.489537    67.428881         1.193617   
min               20.000000   -87.878899   153.000000        11.835737   
25%               39.000000    65.718203   244.000000        15.077516   
50%               44.000000    85.236358   275.500000        15.797984   
75%               51.000000    90.000000   326.000000        16.571982   
max               88.000000    90.000000   512.000000        20.410788   

       blob_centroid_y  blob_height   blob_width  centroid_bottom_is_empty  \
count      1920.000000  1920.000000  1920.000000               1920.000000   
mean         15.380416    29.997917    20.457292                  0.348958   
std           1.869070     0.064533     2.035446                  0.476765   
min          10.582800    28.000000    10.000000                  0.000000   
25%          14.116460    30.000000    20.000000                  0.000000   
50%          15.295089    30.000000    20.000000                  0.000000   
75%          16.354908    30.000000    22.000000                  1.000000   
max          20.841542    30.000000    30.000000                  1.000000   

       centroid_is_empty  centroid_top_is_empty       ...            grid_6_7  \
count        1920.000000            1920.000000       ...         1920.000000   
mean            0.295312               0.211458       ...            2.423958   
std             0.456302               0.408449       ...            4.553251   
min             0.000000               0.000000       ...            0.000000   
25%             0.000000               0.000000       ...            0.000000   
50%             0.000000               0.000000       ...            0.000000   
75%             1.000000               0.000000       ...            2.000000   
max             1.000000               1.000000       ...           16.000000   

          grid_7_0     grid_7_1     grid_7_2     grid_7_3  grid_7_4  \
count  1920.000000  1920.000000  1920.000000  1920.000000      1920   
mean      0.157812     0.149479     0.050521     0.001563         0   
std       1.132490     1.011683     0.464531     0.039508         0   
min       0.000000     0.000000     0.000000     0.000000         0   
25%       0.000000     0.000000     0.000000     0.000000         0   
50%       0.000000     0.000000     0.000000     0.000000         0   
75%       0.000000     0.000000     0.000000     0.000000         0   
max      16.000000    16.000000     8.000000     1.000000         0   

          grid_7_5     grid_7_6     grid_7_7  number_of_holes  
count  1920.000000  1920.000000  1920.000000      1920.000000  
mean      0.022917     0.182292     0.288542         0.377083  
std       0.246911     0.891801     1.539500         0.574330  
min       0.000000     0.000000     0.000000         0.000000  
25%       0.000000     0.000000     0.000000         0.000000  
50%       0.000000     0.000000     0.000000         0.000000  
75%       0.000000     0.000000     0.000000         1.000000  
max       6.000000    12.000000    16.000000         2.000000  

[8 rows x 75 columns]

In [177]:
cols = list(df)
# move the column to head of list using index, pop and insert
cols.insert(len(cols)-1, cols.pop(cols.index('label')))
cols

# use ix to reorder
df = df.loc[:, cols]


AttributeError: 'list' object has no attribute 'loc'

In [ ]:
# #### naar feature analysis verplaatsen #######

# show labels with the holes and amount
df.groupby(['label', 'number_of_holes']).size()

In [176]:
df

['blob_amount_contours',
 'blob_angle',
 'blob_area',
 'blob_centroid_x',
 'blob_centroid_y',
 'blob_height',
 'blob_width',
 'centroid_bottom_is_empty',
 'centroid_is_empty',
 'centroid_top_is_empty',
 'grid_0_0',
 'grid_0_1',
 'grid_0_2',
 'grid_0_3',
 'grid_0_4',
 'grid_0_5',
 'grid_0_6',
 'grid_0_7',
 'grid_1_0',
 'grid_1_1',
 'grid_1_2',
 'grid_1_3',
 'grid_1_4',
 'grid_1_5',
 'grid_1_6',
 'grid_1_7',
 'grid_2_0',
 'grid_2_1',
 'grid_2_2',
 'grid_2_3',
 'grid_2_4',
 'grid_2_5',
 'grid_2_6',
 'grid_2_7',
 'grid_3_0',
 'grid_3_1',
 'grid_3_2',
 'grid_3_3',
 'grid_3_4',
 'grid_3_5',
 'grid_3_6',
 'grid_3_7',
 'grid_4_0',
 'grid_4_1',
 'grid_4_2',
 'grid_4_3',
 'grid_4_4',
 'grid_4_5',
 'grid_4_6',
 'grid_4_7',
 'grid_5_0',
 'grid_5_1',
 'grid_5_2',
 'grid_5_3',
 'grid_5_4',
 'grid_5_5',
 'grid_5_6',
 'grid_5_7',
 'grid_6_0',
 'grid_6_1',
 'grid_6_2',
 'grid_6_3',
 'grid_6_4',
 'grid_6_5',
 'grid_6_6',
 'grid_6_7',
 'grid_7_0',
 'grid_7_1',
 'grid_7_2',
 'grid_7_3',
 'grid_7_4',
 'gri